In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

data = yf.download(tickers = "SPY AAPL MSFT GOOG TSLA VTI TLT",  
            period = "5y",         
            interval = "1d",       
            ignore_tz = True,      
            prepost = True) 


In [ ]:
data = data["Adj Close"]
print(len(data))
new_df=pd.DataFrame()
tickers = ["AAPL","GOOG" , "MSFT", "SPY", "TLT", "TSLA", "VTI"]
print(tickers)

In [ ]:
stock_dividends = [yf.Ticker(ticker).dividends for ticker in tickers]

In [ ]:
for i,k in enumerate(data.columns):
    print(k,i)
    print(data[k].values)
    new_df[tickers[i]] = data[k].values

In [ ]:
new_df

In [ ]:
number_of_assets = 7
returns = new_df.pct_change()

returns = returns.iloc[1:]

mean_daily_returns = returns.mean()
cov_matrix = returns.cov()
print(returns)
def rand_weights(n):
    
    k = np.random.rand(n)
    return k / sum(k)

weights = rand_weights(number_of_assets)

portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)

portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)



In [ ]:

num_portfolios = 300
simulated_returns = []
simulated_standard_deviation = []

for i in range(num_portfolios):
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))

    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    simulated_returns.append(np.array(mu)[0]*252)
    simulated_standard_deviation.append(np.array(sigma)[0]*252)
    
fig = plt.figure()
plt.plot(simulated_returns, simulated_standard_deviation, 'o', markersize=5)
plt.xlabel('Standard Deviation')
plt.ylabel('Mean Return')
plt.title('300 Randomly Generated Markowitz Portfolios')
plt.savefig('Markowitz1.png')

In [ ]:
def dividend_yields(stocks,dividends):
    #annual dividend yield
    yeilds = []
    for i,k in enumerate(stocks):
        market_price = stocks[k].iloc[-1]
        annual_dividends = sum(dividends[i].iloc[-4:])
        div_yield = (annual_dividends/market_price)
        yeilds.append(div_yield)
    return yeilds
print(dividend_yields(new_df,stock_dividends))

In [ ]:
import cvxpy as cp


def Portfolio_Optimization(stocks, dividends,desired_div_yeild,max_variance,desired_return):
    x = cp.Variable(len(stocks.columns))
    yeilds = cp.Parameter(shape=len(stocks.columns), name="yeilds")
    min_return = cp.Parameter(shape=len(stocks.columns), name="min_return")
    max_var = cp.Parameter(shape=len(stocks.columns), name="max_var")
    constraints = []
    constraints.append(cp.sum(x) == 1)  # sum of all allocations should be 1
    constraints.append(cp.sum(yeilds*x) >= desired_div_yeild) # calculate total dividend yield and enforce >= 3%
    constraints.append(cp.sum(min_return*x) >= desired_return)
    constraints.append(cp.sum(max_var*x) <= max_variance)
    constraints.append(x >= 0)
    
    returns = stocks.pct_change()
    returns = returns.iloc[1:]
    weights = rand_weights(number_of_assets)
    
    yearly_returns = returns.mean()*252
    cov_matrix = returns.cov()
    portfolio_var = weights.T @ cov_matrix @ weights
    portfolio_std = np.sqrt(portfolio_var)
    annual_var = np.var(returns) * 252
    
    dividend_annual = sum(dividend_yields(stocks,dividends))/len(dividend_yields(stocks,dividends))
    
    
    yeilds.value = dividend_yields(stocks,dividends)
    min_return.value = np.array([i for i in yearly_returns])
    max_var.value = np.array([var for var in annual_var])
    
    expected_returns = sum([i for i in yearly_returns])/len(yearly_returns)
    objective = cp.Maximize(expected_returns * portfolio_std *dividend_annual)

    problem = cp.Problem(objective, constraints)
    problem.solve()

    if problem.status == 'optimal':
        allocations = x.value
        portfolio_returns = np.dot(allocations, yearly_returns)
        portfolio_risk = np.sqrt(np.dot(allocations.T, np.dot(cov_matrix, allocations)))
        portfolio_dividends = np.dot(allocations, yeilds.value)
        return allocations, portfolio_returns, portfolio_risk, portfolio_dividends
    else:
        print("Optimizations not possivle given constraints")
        return None, None, None, None

In [ ]:
allocations,portfolio_returns,porfolio_risk,portfolio_div = Portfolio_Optimization(new_df,stock_dividends,.015,.2,.015)
print(allocations)
print(portfolio_returns)
print(porfolio_risk)
print(portfolio_div)

In [ ]:
def generate_portfolios(stocks, dividends):
    desired_div_yeild,max_variance,desired_return  = np.linspace(0,.015,20),np.linspace(0,.3,20),np.linspace(0,.5,20)
    df = pd.DataFrame()
    allocations = []
    returns = []
    risk = []
    divs = []
    for div in desired_div_yeild:
        print(div)
        for var in max_variance:          
            for ret in desired_return:
                try:
                    allocation,portfolio_returns,porfolio_risk,portfolio_div = Portfolio_Optimization(stocks, dividends,div,var,ret)
                    if len(allocation)>0:
                        allocations.append(allocation)
                        returns.append(portfolio_returns)
                        risk.append(porfolio_risk)
                        divs.append(portfolio_div)
                    else:
                        break
                except:
                    break
    df = pd.DataFrame()      
    df["Returns"] = returns
    df["Risk"] = risk
    df["divs"] = divs
    return df

In [ ]:
results = generate_portfolios(new_df,stock_dividends)
results

In [ ]:
plt.scatter(results["Risk"],results["Returns"] +results["divs"])
plt.xlabel('Standard Deviation')
plt.ylabel('Mean Yearly Return')
plt.title('4182 Generated Optimized Portfolios')
plt.savefig('Final.png')
